<a href="https://colab.research.google.com/github/MRyo-ie/DataMining_Report3_Lv4_NLP_Models/blob/GoogleColab-merge/Report3_Lv_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Report3

## Level 4: 文書分類せよ。


In [0]:
# 自分の Google Drive をマウント
# https://qiita.com/uni-3/items/201aaa2708260cc790b8#drive内のディレクトリをマウントする220180920
from google.colab import drive
import os
drive.mount('/content/gdrive')

#@markdown ### GitHubリポジトリをcloneして保存するパス （Google Drive上）
REPOSITORY_ROOT_DIR = 'Colab Notebooks/learning/old_nlp_techs' #@param {type: "string"}

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### <<  実行手順  >>

1.   このボタンを押す　→　　[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)
1. ↖︎（左上）のどこかに、「ドライブに保存」があるので、それをクリックして、  
  自分の Google Drive にこのファイルをコピーする。
1. ↑ の変数を設定する。
1. 実行。  
  （↑で GoogleDrive をマウントするので、（英語の）指示に従って、  
   「ログイン」→「パスコードをコピー」→「四角枠に貼り付けしてEnter」）
1. 後は勝手に実行してくれるはず(笑)

### 【準備】 データ
今回は livedoor の記事データセットを使用。


In [0]:
import configparser
import glob
import os, sys
import pandas as pd
import subprocess
import tarfile
from tqdm import tqdm
from urllib.request import urlretrieve

In [0]:
REPOSITORY_URL = ''
DATA_DL_URL = "https://www.rondhuit.com/download/ldcc-20140209.tar.gz"
GDRIVE_ROOT = '/content/gdrive/My Drive/'
REPO_PATH = os.path.join(GDRIVE_ROOT, REPOSITORY_ROOT_DIR)

# よく使うパス
DATA_DIR = os.path.join(REPO_PATH, 'datas', 'articles_livedoor')
TMP_DIR = os.path.join(DATA_DIR, 'tmp')
FILE_PATH = os.path.join(DATA_DIR, 'tmp', DATA_DL_URL.split('/')[-1])

print("""
------------------------
<<<  パス一覧  >>>
REPO_PATH = {}

DATA_DL_URL = {}
DATA_DIR = {}
TMP_DIR = {}
FILE_PATH = {}
""".format(REPO_PATH, DATA_DL_URL, DATA_DIR, TMP_DIR, FILE_PATH))


FILEURL = https://www.rondhuit.com/download/ldcc-20140209.tar.gz
FILEPATH = /content/gdrive/My Drive/
EXTRACTDIR = /content/gdrive/My Drive/Colab Notebooks/datasets/[NLP]「コーパス」/livedoor/ldcc-20140209.tar.gz

REPOSITORY_ROOT_DIR = /content/gdrive/My Drive/Colab Notebooks/datasets/[NLP]「コーパス」/livedoor



#### 読み込み

In [0]:
# 本家データセット に更新があった場合
is_renew_file = False

%cd  "$TMP_DIR/"
if not os.path.exists(FILE_PATH) or is_renew_file:
    ! wget  "$DATA_DL_URL"

if is_renew_file:
    # 時間かかる。
    !tar  -zxvf   'ldcc-20140209.tar.gz'
    !ls

/content/gdrive/My Drive/Colab Notebooks/datasets/[NLP]「コーパス」/livedoor


In [0]:
categories = [ 
    name for name 
    in os.listdir( os.path.join(TMP_DIR, "text") ) 
    if os.path.isdir( os.path.join(TMP_DIR, "text", name) ) ]

categories = sorted(categories)
categories

#### データの抽出・整備



- ラベル（記事の種類）一覧

In [0]:
table = str.maketrans({
    '\n': '',
    '\t': '　',
    '\r': '',
})

def extract_txt(filename):
    with open(filename) as text_file:
        # 0: URL, 1: timestamp
        text = text_file.readlines()[2:]
        text = [sentence.strip() for sentence in text]
        text = list(filter(lambda line: line != '', text))
        return ''.join(text)

%cd  "$TMP_DIR/"

/content/gdrive/My Drive/Colab Notebooks/datasets/[NLP]「コーパス」/livedoor


In [0]:
%cd  "$TMP_DIR/"
all_text = []
all_label = []

# 割と時間かかる。
for cat in tqdm(categories):
    files = glob.glob(os.path.join("text", cat, "{}*.txt".format(cat)))
    files = sorted(files)
    body = [ extract_txt(elem).translate(table) for elem in files ]
    label = [cat] * len(body)
    
    all_text.extend(body)
    all_label.extend(label)

print(all_label)

















  0%|          | 0/9 [00:00<?, ?it/s]

/content/gdrive/My Drive/Colab Notebooks/datasets/[NLP]「コーパス」/livedoor


















 11%|█         | 1/9 [00:01<00:10,  1.26s/it]















 22%|██▏       | 2/9 [00:02<00:08,  1.15s/it]















 33%|███▎      | 3/9 [00:02<00:06,  1.05s/it]















 44%|████▍     | 4/9 [01:15<01:52, 22.47s/it]

KeyboardInterrupt: ignored

- 特徴量とラベル

In [0]:
df = pd.DataFrame({'text' : all_text, 'label' : all_label})
df.head()

In [0]:
df = df.sample(frac=1, random_state=23).reset_index(drop=True)
df.head()

- データを csv に保存する。
  (機械学習モデル用)

In [0]:
df[:len(df) // 5].to_csv( os.path.join(TMP_DIR, 'extracts', "test.tsv"), sep='\t', index=False)
df[len(df) // 5:len(df)*2 // 5].to_csv( os.path.join(TMP_DIR, 'extracts', "dev.tsv"), sep='\t', index=False)
df[len(df)*2 // 5:].to_csv( os.path.join(TMP_DIR, 'extracts', "train.tsv"), sep='\t', index=False)

### Layer1 : 分かち書き（Tokenize）
- 今回は Sentence Piece を使ってみる。

In [0]:
%cd  "$REPOSITORY_ROOT_DIR"

from WikiIterator import wiki_sentences
# データを用意する。学習用、検証用、テスト用にそれぞれ24k, 3k, 3k文。
train_size = 24000
valid_size = 3000
test_size = 3000

gen = wiki_sentences()
for filename, size in zip(
    ['sp/wiki_train.txt', 'sp/wiki_valid.txt', 'sp/wiki_test.txt'],
    [train_size, valid_size, test_size]):
    
    i = 0
    output = ''
    for text in gen:
        output += text + '\n'
        i += 1
        if i >= size:
            with open(filename,  'w', encoding='utf-8') as f:
                f.write(output)
            break

In [0]:
# 下記コマンドを走らせてSentencePieceのモデルを用意しておく
!spm_train --input=sp/wiki_train.txt --model_prefix=sp/sp --vocab_size=8000

In [0]:
# 学習したSentencePieceモデルを読み込む
import sentencepiece as spm
sp = spm.SentencePieceProcessor()
sp.Load('sp/sp.model')

In [0]:
# コーパスの頭を覗き見てみる
i = 0
for text in wiki_sentences():
    print(' '.join(sp.EncodeAsPieces(text)))
    i += 1
    if i >= 5: break

In [0]:
import torch
import torch.nn as nn
from torch.autograd import Variable

import numpy as np
import pandas as pd
import glob
import os
import time
import math

use_gpu = torch.cuda.is_available()

In [0]:
# wakati_corpus.txtから一部データを読み込む
class Corpus(object):
    def __init__(self):
        self.sp = spm.SentencePieceProcessor()
        self.sp.Load('sp/sp.model')

        self.train = self.load_from_file('sp/wiki_train.txt')
        self.valid = self.load_from_file('sp/wiki_valid.txt')
        self.test = self.load_from_file('sp/wiki_test.txt')

        self.train = self.assign_ids(self.train)
        self.valid = self.assign_ids(self.valid)
        self.test = self.assign_ids(self.test)
    
    def load_from_file(self, filename):
        with open(filename, 'r', encoding='utf-8') as f:
            return f.read().split('\n')
    
    def assign_ids(self, texts):
        tokens = []
        for text in texts:
            tokens.extend(self.sp.EncodeAsIds(text))
        
        ids = torch.from_numpy(np.array(tokens, dtype=np.int64))
        return ids

In [0]:
# コーパスを実際に読み込む
corpus = Corpus()